In [ ]:
import os
import matplotlib.pyplot as plt
import h5py
import numpy as np
import networkx as nx
import yaml
from numpy.typing import NDArray
from sklearn.cluster import SpectralClustering

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "font.size": 20,
})

def get_adj_matrix(
    nodes_number: int,
    pairs: NDArray,
) -> NDArray:
    adj_matrix = np.zeros((nodes_number, nodes_number), dtype=np.float64)
    for pair in pairs:
        adj_matrix[pair[0], pair[1]] = 1
        adj_matrix[pair[1], pair[0]] = 1
    return adj_matrix

script_dir = os.getcwd()
# ---------- here is the path to the results directory ----------------------
# here is the path to the results directory
# ---------------------------------------------------------------------------
results_dir_path = f"{script_dir}/outputs/qbp/very_large/2024-05-16/13-15-25"
results_path = f"{results_dir_path}/result"
heuristics_results_path = f"{results_dir_path}/mqlib_result"
config_path = f"{results_dir_path}/.hydra/config.yaml"
with open(config_path, "r") as f:
    raw_config = f.read()
    config = yaml.safe_load(raw_config)
sample_measurements = bool(config["quantum_annealing_schedule"]["sample_measurements"])
layers_number = int(config["quantum_annealing_schedule"]["steps_number"])
time_step = float(config["quantum_annealing_schedule"]["total_time_step_size"])
time = np.arange(layers_number) * time_step
best_heuristic_energy = 1e10
best_heuristic_name = None
best_heuristic_configuration = None
with h5py.File(heuristics_results_path, "r") as f:
    for key in f.keys():
        if "energy" in key:
            energy = np.array(f[key])
            if energy < best_heuristic_energy:
                best_heuristic_energy = energy
                best_heuristic_name = key.split("_")[0]
                best_heuristic_configuration = np.array(f[f"{best_heuristic_name}_configuration"])
best_heuristic_color = list(map(lambda x: "indigo" if x > 0 else "indianred", np.array(best_heuristic_configuration)))
with h5py.File(results_path, "r") as f:
    circuit = nx.Graph()
    fields = np.array(f["fields"])
    pairs = np.array(f["coupled_spin_pairs"])
    nodes_number = fields.shape[0]
    edges_number = pairs.shape[0]
    coupling_amplitudes = np.array(f["coupling_amplitudes"])
    truncation_errors = np.array(f["truncation_errors"])
    entropies = np.array(f["entropies"])
    entropies = entropies.reshape((-1, edges_number))
    mismatching_nodes_number = 0
    quantum_annealing_color_wo_sampling = list(map(lambda x: "indigo" if x > 0.5 else "indianred", np.array(f["density_matrices"])[:, 0, 0]))
    quantum_annealer_energy_wo_sampling =  np.array(f["quantum_annealer_energy_wo_sampling"])
    if sample_measurements:
        quantum_annealer_coloring = list(map(lambda x: "indigo" if x > 0.5 else "indianred", np.array(f["quantum_annealer_configuration"])))
        quantum_annealer_energy = np.array(f["quantum_annealer_energy"])
    for i in range(len(quantum_annealing_color_wo_sampling)):
        if quantum_annealing_color_wo_sampling[i] != best_heuristic_color[i]:
            quantum_annealing_color_wo_sampling[i] = "cyan"
            best_heuristic_color[i] = "cyan"
            mismatching_nodes_number += 1
    vidal_distances_after_regauging = np.array(f["vidal_distances_after_regauging"])
    for id in range(fields.shape[0]):
        circuit.add_node(id)
    for pair in pairs:
        circuit.add_edge(pair[0], pair[1])
    print(f"Best heuristic energy is is {best_heuristic_energy}")
    print(f"Best heuristic name: {best_heuristic_name}")
    fig_graph_best_heuristic, ax = plt.subplots(1, 1, figsize=(5, 5))
    #ax.set_title(r"${\rm " + best_heuristic_name + r"}$")
    nx.draw_kamada_kawai(circuit, node_size=8, width=1., edge_color='silver', node_color=best_heuristic_color, ax=ax)
    fig_graph_wo_sampling, ax = plt.subplots(1, 1, figsize=(5, 5))
    #ax.set_title(r"${\rm Quantum \ Annealing}$")
    print(f"Quantum annealing energy wo sampling is {quantum_annealer_energy_wo_sampling}")
    nx.draw_kamada_kawai(circuit, node_size=8, width=1., edge_color='silver', node_color=quantum_annealing_color_wo_sampling, ax=ax)
    if sample_measurements:
        fig_graph_with_sampling, ax = plt.subplots(1, 1, figsize=(5, 5))
        print(f"Quantum annealing energy with sampling is {quantum_annealer_energy}")
        nx.draw_kamada_kawai(circuit, node_size=8, width=1., edge_color='silver', node_color=quantum_annealer_coloring, ax=ax)
    fig_truncation_error, ax = plt.subplots(1, 1)
    ax.plot(1 - ((1 - truncation_errors ** 2) ** 2).cumprod(), 'b')
    ax.set_ylabel(r"$1 - F$")
    ax.set_xlabel(r"${\rm Two-qubit \ gate \ number}$")
    ax.set_xticks([0, 500000, 1000000, 1500000, 2000000])
    ax.set_xticklabels([r"$0$", r"$5\cdot 10^5$", r"$10^6$", r"$1.5 \cdot 10^6$", r"$2 \cdot 10^6$"])
    ax.set_yscale('log')
    #ax.set_xscale('log')
    fig_vidal_dist, ax = plt.subplots(1, 1)
    ax.plot(vidal_distances_after_regauging[-nodes_number:], 'b')
    ax.set_yscale("log")
    ax.set_ylabel(r"{\rm Vidal distance}")
    ax.set_xlabel(r"{\rm Regauging number}")
    fig_entropy, ax = plt.subplots(1, 1)
    sc = SpectralClustering(
        affinity='precomputed',
        random_state=0,
        n_clusters=2,
    )
    adj_matrix = get_adj_matrix(nodes_number, pairs)
    clustering = sc.fit(adj_matrix).labels_
    cut_entropy = np.zeros((entropies.shape[0],))
    cut_size = 0
    for pair, entropy in zip(pairs, entropies.T):
        if clustering[pair[0]] != clustering[pair[1]]:
            cut_entropy += entropy
            cut_size += 1
    ax.plot(time, cut_entropy, 'b')
    ax.set_ylabel(r"$S_A$")
    ax.set_xlabel(r"${\rm Time}$")
    print(f"Edges_number: {edges_number}")
    print(f"Layers number: {layers_number}")
    print(f"Mismatching nodes number: {mismatching_nodes_number}")
    print(f"Cut size: {cut_size}")
    print(f"Parts sizes: {clustering.sum()}, {nodes_number - clustering.sum()}")
    print(f"Min bond dimension: {np.exp(np.max(cut_entropy))}")

In [ ]:
fig_graph_best_heuristic.savefig(f"{results_dir_path}/best_heuristic_graph.pdf", bbox_inches="tight")
fig_graph_wo_sampling.savefig(f"{results_dir_path}/qbp_graph_wo_sampling.pdf", bbox_inches="tight")
fig_truncation_error.savefig(f"{results_dir_path}/qbp_infidelity.pdf", bbox_inches="tight")
fig_vidal_dist.savefig(f"{results_dir_path}/qbp_vidal_distance.pdf", bbox_inches="tight")
fig_entropy.savefig(f"{results_dir_path}/qbp_entropy.pdf", bbox_inches="tight")
if sample_measurements:
    fig_graph_with_sampling.savefig(f"{results_dir_path}/qbp_graph_with_sampling.pdf", bbox_inches="tight")